In [1]:
%run mt_datagen.ipynb
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import coo_matrix as sparse
import torch
from torch.nn import Softmax as soft
import torch.nn.functional as F
from torch_geometric.nn import TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch.nn import Linear
from torch_geometric.nn import GCNConv, GravNetConv, PANConv, PANPooling, ASAPooling, SAGPooling, ChebConv, GraphConv,GATv2Conv
from torch_geometric.nn import GatedGraphConv 
from torch_geometric.nn import ResGatedGraphConv
from torch_geometric.nn import SAGEConv, GraphConv, GINConv, BatchNorm,  DynamicEdgeConv
from torch_geometric.data import Data
from torch_geometric.datasets import TUDataset  
import torch_geometric as tg
from torch_geometric.data import DataLoader
import random
from torchsummary import summary
print('GPU backend =', torch.cuda.is_available())
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

GPU backend = True


In [2]:
class model:
    def __init__(self):
        self.N = 1000
        self.cut = 0.9
        self.epoch = 50
        self.batch_size = 200

        self.batch_amount = 2000 # batch_amount = batches*CPU_threads
        self.graph_type = "ER" #graph topology
        self.CPU_threads = 8 #number of threads
        self.nodes_N = 400 #number of simulated nodes
        self.iterations = 50
        self.infection_probability = 1/35
        self.removal_probability = 1/40
        self.latent_period = 1/30
        self.p = 4 #mean number of edges per node
        self.one_hot_features = True
        self.two_d_labels = False
        self.show_progress = False
        self.types = 'SEIR'
        self.filename = '' #leave empty for automatic generated
        self.save = False
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def generator(self):
        # execute datagen from mt_datagen.ipynb
        data = datagen(batch_amount = self.batch_amount, graph_type = self.graph_type,p = self.p/self.nodes_N , CPU_threads = self.CPU_threads, N = self.nodes_N, 
                iterations = self.iterations, infection_probability = self.infection_probability, removal_probability = self.removal_probability, 
                latent_period = self.latent_period, filename = self.filename, one_hot_features = self.one_hot_features, two_d_labels = self.two_d_labels, 
                show_progress = self.show_progress, types=self.types,save=self.save)
        #generate data given the parameters
        data.generate()
        self.dataset = data.show()
        sims = len(dataset)
        trainingsset, testset = dataset[:int(sims*self.cut)], dataset[int(sims*self.cut):]
        # pack simulations into mini-batch 
        self.trainingsset = DataLoader(trainingsset, batch_size=self.batch_size, shuffle=True, drop_last=False)
        self.testset = DataLoader(testset, batch_size=1, shuffle=True, drop_last=False)
        print('dataset ready for deployment.')

    def train(self):
        # train network, print accuracy at the end
        self.model0 = self.Net1().to(self.device)
        self.optimizer0 = torch.optim.Adam(self.model0.parameters(), lr=0.01)
        self.loss0 = torch.nn.CrossEntropyLoss()
        threshold = 0.0

        self.train_network()

        #del trainingsset

        self.model0.eval()
        #perm = list(range(len(testset)))
        #random.shuffle(perm)
        #testset = [testset[index] for index in perm]

        top5 = 0
        top1 = 0
        all1 = 0
        prog = 0
        
        print("testset:")
        for it, data in enumerate(self.testset):
            test = self.model0(data.to(self.device)).detach().to('cpu').numpy()
            max1 = np.argsort(test[:,1])[-5:]

            if np.argmax(data.y.detach().to('cpu').numpy()) in max1:
                top5+=1
                if np.argmax(data.y.detach().to('cpu').numpy()) == max1[-1]:
                    top1+=1

            all1 +=1

            if it/len(self.testset)*100>=prog:
                print(prog, "%")
                prog+=10

        print("top 5 Accuracy: ", top5/all1*100, "%")
        print("top 1 Accuracy: ", top1/all1*100, "%")
        if self.save == True: torch.save(self.model0, 'model')
            
        print("trainingset (testing for overfitting):")
        for it, data in enumerate(self.trainingsset[1000]):
            test = self.model0(data.to(self.device)).detach().to('cpu').numpy()
            max1 = np.argsort(test[:,1])[-5:]

            if np.argmax(data.y.detach().to('cpu').numpy()) in max1:
                top5+=1
                if np.argmax(data.y.detach().to('cpu').numpy()) == max1[-1]:
                    top1+=1

            all1 +=1

            if it/10>=prog:
                print(prog, "%")
                prog+=10

        print("top 5 Accuracy: ", top5/all1*100, "%")
        print("top 1 Accuracy: ", top1/all1*100, "%")


    # wrapper to measure execution time of function
    def runtime(func): 
        def wrapper(*args, **kargs):
            print("\033[1mExecuting function", func.__name__,":\n\033[0m")
            start = t.time()
            result = func(*args, **kargs)
            print ("\033[1m\nFinished execution of function {0} in {1:.5f} seconds.\n\033[0m".format(func.__name__, (t.time()-start)))
            return result
        return wrapper

    #onehot -> non-onehot
    def from_one_hot(dataset):
        prog = 0
        l = len(dataset)
        for it, data in enumerate(dataset):
            if data.y.ndim != 2:
                print("data {} does not have 2 dims... skipping".format(it))
            else:
                data.y = data.y[:,1]
        print("dataset now converted from one-hot")
    
    # print status of current dataset
    def model_status(self):
        sus, exp, inf, rem = ([] for i in range(4))
        for i, d in enumerate(self.dataset):
            d = data[i]['x']
            sus.append(np.count_nonzero(d[:,0]))
            inf.append(np.count_nonzero(d[:,1]))
            exp.append(np.count_nonzero(d[:,2]))
            rem.append(np.count_nonzero(d[:,3]))
        status = np.vstack((sus,exp,inf,rem))
        maxrem = np.max(status[3])
        status = np.mean(status, axis=1)
        status = np.append(status,maxrem)
        print(f'#Susceptible = {status[0]}\n#Exposed = {status[1]}\n#Infected = {status[2]}\n#Removed = {status[3]}\nmax # of removed = {status[4]}')
    
    # train function
    def train_network(self):
        self.model0.train()
        for i in range(self.epoch):
            loss_sum = np.array([])
            for data in self.trainingsset:
                data.to(self.device)
                self.optimizer0.zero_grad()
                out = self.model0(data)
                y = data.y
                loss = self.loss0(out, y.type(torch.long))
                loss_sum = np.append(loss_sum,loss.item())
                loss.backward()
                self.optimizer0.step()
            print(f'loss for epoch {i} = ',np.mean(loss_sum))

        return self.model0, self.optimizer0


    # different models we have tested. Final model is 'Net1'
    class u_net_asap(torch.nn.Module):
        def __init__(self):
            super().__init__()
            normalize = True
            size = 20
            self.sagec1 = ResGatedGraphConv(4,size) #[20,200]
            self.sagp1 = ASAPooling(size)
            self.bn1 = BatchNorm(size)

            self.sagec2 = ResGatedGraphConv(size,size)
            self.sagp2 = ASAPooling(size)
            self.bn2 = BatchNorm(size)
            self.sagec3 = ResGatedGraphConv(size,size)


            self.up1 = ResGatedGraphConv(size,size*2)

            self.up2 = ResGatedGraphConv(size,size*2)

            self.sagec7 = ResGatedGraphConv(size,size)
            self.classifier = Linear(size,2)



        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch
            x = self.sagec1(x, edge_index) #first layer sage
            x = F.relu(x)
            x = F.dropout(x, p=p)
            #first downsampling layer
            x1, edge_index1, _, batch1, _ = self.sagp1(x, edge_index, batch=batch)
            x1 = self.sagec2(x1,edge_index1) #simple u-net
            x1 = self.bn1(x1)
            x1 = F.relu(x1)
            x1 = F.dropout(x1, p=p)
            #second downsampling layer
            x2, edge_index2, _, batch2, _ = self.sagp1(x1, edge_index1, batch=batch1)
            x2 = self.sagec2(x2,edge_index2) #simple u-net
            x2 = self.bn2(x2)
            x2 = F.relu(x2)
            x2 = F.dropout(x2, p=p)
            #first upscaling layer
            x2 = self.up1(x2, edge_index2)
            x2 = F.relu(x2)
            x1 = x1+x2.reshape(x1.shape)
            x1 = F.dropout(x1, p=p)
            #second upscaling layer
            x1 = self.up2(x1, edge_index1)
            x1 = F.relu(x1)
            x = x+x1.reshape(x.shape)
            x = F.dropout(x, p=p)
            #last concetenated layer
            x = self.sagec3(x,edge_index) #third layer sage
            x = F.relu(x)
            x = F.dropout(x, p=p)
            #classification layer                 
            x = self.classifier(x)
            x = F.log_softmax(x,dim=1)
            return x

    class res_gat_net(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.sconv1 = ResGatedGraphConv(4, 20)
            self.resgatenn = ResGatedGraphConv(20, 20)
            self.sconv2 = ResGatedGraphConv(20, 20)
            self.sconv3 = ResGatedGraphConv(20, 20)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x,dim=1)
            return x


    class gatv2_conv(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.sconv1 = GATv2Conv(4, 20)
            self.resgatenn = GATv2Conv(20, 20)
            self.sconv2 = GATv2Conv(20, 20)
            self.sconv3 = GATv2Conv(20, 20)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x,dim=1)
            return x

    class pure_sage(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.sconv1 = SAGEConv(4, 20)
            self.resgatenn = SAGEConv(20, 20)
            self.sconv2 = SAGEConv(20, 20)
            self.sconv3 = SAGEConv(20, 20)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x,dim=1)
            return x

    class graph_conv(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.sconv1 = GraphConv(4, 20)
            self.resgatenn = GraphConv(20, 20)
            self.sconv2 = GraphConv(20, 20)
            self.sconv3 = GraphConv(20, 20)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x,dim=1)
            return x

    class chebby_net(torch.nn.Module):
        def __init__(self):
            super().__init__()
            k = 3
            self.sconv1 = ChebConv(4, 20,k)
            self.resgatenn = ChebConv(20, 20,k)
            self.sconv2 = ChebConv(20, 20,k)
            self.sconv3 = ChebConv(20, 20,k)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x)
            return x

    #define Net
    class sage_res(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.sconv1 = SAGEConv(4, 20)
            self.resgatenn = ResGatedGraphConv(20, 20)
            self.sconv2 = SAGEConv(20, 20)
            self.sconv3 = SAGEConv(20, 20)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x,dim=1)
            return x
    #define Net
    class gcn(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.sconv1 = GCNConv(4, 20)
            self.resgatenn = GCNConv(20, 20)
            self.sconv2 = GCNConv(20, 20)
            self.sconv3 = GCNConv(20, 20)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x,dim=1)
            return x

    class Net1(torch.nn.Module):
        def __init__(self):
            super().__init__()
            self.sconv1 = ChebConv(1, 20,3)
            self.resgatenn = ChebConv(20, 20,3)
            self.sconv2 = ResGatedGraphConv(20, 20)
            self.sconv3 = ResGatedGraphConv(20, 20)
            self.classifier1 = Linear(20,2)


        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sconv1(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.resgatenn(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv2(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.sconv3(x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, training=self.training, p=p)
            x = self.classifier1(x)
            x = F.log_softmax(x,dim=1)
            return x

    #define Net
    class simple_u(torch.nn.Module):
        def __init__(self):
            super().__init__()
            normalize = True
            self.sagec1 = SAGEConv(4,20, normalize=normalize) #[20,200]
            self.sagp1 = SAGPooling(20)

            self.sagec2 = SAGEConv(20,40, normalize=normalize)

            self.sagec3 = SAGEConv(20,20, normalize=normalize)
            self.classifier = Linear(20,2)




        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sagec1(x, edge_index) #first layer sage
            x = F.relu(x)
            x = F.dropout(x, p=p)


            x1, edge_index1, weights1, batch1, perm,_ = self.sagp1(x, edge_index,batch=batch)
            x1 = self.sagec2(x1,edge_index1) #simple u-net
            x1 = F.relu(x1)
            x1 = F.dropout(x1, p=p)
            x = x+x1.reshape(x.shape) #concetenate U with first layer
            x = self.sagec3(x,edge_index) #third layer sage
            x = F.relu(x)
            x = F.dropout(x, p=p)

            x = self.classifier(x)
            x = F.log_softmax(x,dim=1)




            return x


    #define Net
    class u_net(torch.nn.Module):
        def __init__(self):
            super().__init__()
            normalize = True
            size = 20
            self.sagec1 = SAGEConv(4,size, normalize=normalize) #[20,200]
            self.sagp1 = SAGPooling(size)
            self.bn1 = BatchNorm(size)

            self.sagec2 = SAGEConv(size,size, normalize=normalize)
            self.sagp2 = SAGPooling(size)
            self.bn2 = BatchNorm(size)

            self.sagec3 = SAGEConv(size,size, normalize=normalize)
            self.sagp3 = SAGPooling(size)
            self.bn3 = BatchNorm(size)

            self.up1 = SAGEConv(size,size*2, normalize=normalize)

            self.up2 = SAGEConv(size,size*2, normalize=normalize)

            self.sagec7 = SAGEConv(size,size, normalize=normalize)
            self.classifier = Linear(size,2)



        def forward(self, data):
            p = 0.1
            x, edge_index, batch = data.x, data.edge_index, data.batch

            x = self.sagec1(x, edge_index) #first layer sage
            x = F.relu(x)
            x = F.dropout(x, p=p)
            #first downsampling layer
            x1, edge_index1, weights1, batch1, perm1,_ = self.sagp1(x, edge_index, batch=batch)
            x1 = self.sagec2(x1,edge_index1) #simple u-net
            x1 = self.bn1(x1)
            x1 = F.relu(x1)
            x1 = F.dropout(x1, p=p)
            #second downsampling layer
            x2, edge_index2, weights2, batch2, perm2,_ = self.sagp1(x1, edge_index1, batch=batch1)
            x2 = self.sagec2(x2,edge_index2) #simple u-net
            x2 = self.bn2(x2)
            x2 = F.relu(x2)
            x2 = F.dropout(x2, p=p)
            #first upscaling layer
            x2 = self.up1(x2, edge_index2)
            x2 = F.relu(x2)
            x1 = x1+x2.reshape(x1.shape)
            x1 = F.dropout(x1, p=p)
            #second upscaling layer
            x1 = self.up2(x1, edge_index1)
            x1 = F.relu(x1)
            x = x+x1.reshape(x.shape)
            x = F.dropout(x, p=p)
            #last concetenated layer
            x = self.sagec3(x,edge_index) #third layer sage
            x = F.relu(x)
            x = F.dropout(x, p=p)
            #classification layer                 
            x = self.classifier(x)
            x = F.log_softmax(x,dim=1)
            return x

In [3]:
# alternative implementation of pytorch_geometric.data.DataLoader with memory handling
class classDataLoader:
    def __init__(self, dataset, batchsize):
        super().__init__()
        self.dataset = dataset
        self.batchsize = batchsize
    
    def __iter__(self):
        self.index=0
        self.batch = DataLoader(self.dataset, self.batchsize)
        return self
    
    def __next__(self):
        if self.index >= batchsize-1:
            raise StopIteration
        return self.batch[self.index]

    def batch(self):
        return DataLoader(self.dataset, self.batchsize)
    
 # "backend" implementation of DataLoader to create actual batches
def DataLoader(dataset, batchsize):
    batch_list = []
    bat = Batch()
    np.random.shuffle(dataset)
    
    N = len(dataset)
    full_batches = int(N/batchsize)
    extra = full_batches*batchsize
    
    for batch_number in range(full_batches):
        bat = Batch()
        data_list = dataset[batchsize*batch_number:batchsize*(batch_number+1)]
        batch_list.append(bat.from_data_list(data_list))
    
    if extra!=N:
        extra_data_list = dataset[extra:]
        batch_list.append(bat.from_data_list(extra_data_list))
    
    return batch_list

# returns dataset from list of batches (dtype Batch -> dtype Data)

def DataUnLoader(batch_list):
    dataset = []
    batch_amount = len(batch_list)
    
    for batch_index in range(batch_amount):
        dataset = dataset + batch_list[batch_index].to_data_list()
    
    return dataset

In [4]:
model = model()
model.generator()
model.train()

2021-10-03 16:36:38,880	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265
(pid=7744) 2021-10-03 16:36:53,989	ERROR serialization.py:256 -- [WinError 1455] The paging file is too small for this operation to complete. Error loading "C:\Users\basti\anaconda3\envs\env\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.
(pid=7744) Traceback (most recent call last):
(pid=7744)   File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 254, in deserialize_objects
(pid=7744)     obj = self._deserialize_object(data, metadata, object_ref)
(pid=7744)   File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 190, in _deserialize_object
(pid=7744)     return self._deserialize_msgpack_data(data, metadata_fields)
(pid=7744)   File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 168, in _deserialize_msgpack_data
(pid=7744)     python_objects = self._deser

RayTaskError: [36mray::SEIR_gen()[39m (pid=7744, ip=10.16.4.194)
  File "python\ray\_raylet.pyx", line 495, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 516, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 385, in ray._raylet.raise_if_dependency_failed
ray.exceptions.RaySystemError: System error: [WinError 1455] The paging file is too small for this operation to complete. Error loading "C:\Users\basti\anaconda3\envs\env\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.
traceback: Traceback (most recent call last):
  File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 254, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
  File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 190, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
  File "C:\Users\basti\anaconda3\envs\env\l

(pid=488) Executing function generate_data :
(pid=488) 


(pid=9820) 2021-10-03 16:36:56,618	ERROR serialization.py:256 -- module 'torch' has no attribute '_utils_internal'
(pid=9820) Traceback (most recent call last):
(pid=9820)   File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 254, in deserialize_objects
(pid=9820)     obj = self._deserialize_object(data, metadata, object_ref)
(pid=9820)   File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 190, in _deserialize_object
(pid=9820)     return self._deserialize_msgpack_data(data, metadata_fields)
(pid=9820)   File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 168, in _deserialize_msgpack_data
(pid=9820)     python_objects = self._deserialize_pickle5_data(pickle5_data)
(pid=9820)   File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 158, in _deserialize_pickle5_data
(pid=9820)     obj = pickle.loads(in_band)
(pid=9820)   File "C:\Users\basti\anaconda3\envs\

(pid=40148) Executing function generate_data :
(pid=40148) 


2021-10-03 16:37:01,940	ERROR worker.py:79 -- Unhandled error (suppress with RAY_IGNORE_UNHANDLED_ERRORS=1): ray::SEIR_gen() (pid=9820, ip=10.16.4.194)
  File "python\ray\_raylet.pyx", line 495, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 516, in ray._raylet.execute_task
  File "python\ray\_raylet.pyx", line 385, in ray._raylet.raise_if_dependency_failed
ray.exceptions.RaySystemError: System error: module 'torch' has no attribute '_utils_internal'
traceback: Traceback (most recent call last):
  File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 254, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
  File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 190, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
  File "C:\Users\basti\anaconda3\envs\env\lib\site-packages\ray\serialization.py", line 168, in _deserialize_msgpack_dat

(pid=488) 
(pid=488) Finished execution of function generate_data in 113.44862 seconds.
(pid=488) 
(pid=40148) 
(pid=40148) Finished execution of function generate_data in 113.15056 seconds.
(pid=40148) 


# Aditional Data Manipulation and Visualization

In [ ]:
def x_to_one_hot(dataset, features=4):
    for it, data in enumerate(dataset):
        one_hot = torch.zeros((data.x.shape[0], features))
        one_hot[np.arange(data.x.shape[0], dtype=int), data.x[:,0].long()]= 1
        data.x = one_hot.to(device)

In [5]:
# print histogram of one hot data distribution
def datahist(dataset, SEIR=True):
    susceptible = np.zeros(400)
    infected = np.zeros(400)
    removed = np.zeros(400)
    
    s, e, i, r = [], [], [], []
    
    if SEIR:
        for data in dataset:
            s.append(int(np.sum(data.x.cpu().numpy()[:,0])))
            e.append(int(np.sum(data.x.cpu().numpy()[:,1])))
            i.append(int(np.sum(data.x.cpu().numpy()[:,2])))
            r.append(int(np.sum(data.x.cpu().numpy()[:,3])))
        
        plt.title("susceptible")
        plt.hist(s, bins=70)
        #plt.savefig("WS1")
        plt.show()
        plt.title("exposed")
        plt.hist(e, bins=70)
        #plt.savefig("WS2")
        plt.show()
        plt.title("infected")
        plt.hist(i, bins=70)
        #plt.savefig("WS3")
        plt.show()
        plt.title("removed")
        plt.hist(r, bins=70)
        #plt.savefig("WS4")
        plt.show()
    
    else:
        for data in dataset:
            s.append(int(np.sum(data.x.cpu().numpy()[:,0])))
            i.append(int(np.sum(data.x.cpu().numpy()[:,1])))
            r.append(int(np.sum(data.x.cpu().numpy()[:,2])))
        
        plt.title("susceptible")
        plt.hist(s, bins=70)
        #plt.savefig("er_sir1.jpg")
        plt.show()
        plt.title("infected")
        plt.hist(i, bins=70)
        #plt.savefig("er_sir2.jpg")
        plt.show()
        plt.title("removed")
        plt.hist(r, bins=70)
        #plt.savefig("er_sir3.jpg")
        plt.show()

In [6]:
# this function can be used to remove graphs where only one node is not susceptible
def onenodefilter(dataset, total=400):
    newdataset = dataset
    simple = []
    
    for i, data in enumerate(dataset):
         if torch.sum(data.x[:,0]).item() == total-1:
            simple.append(i)

    for s in simple[::-1]:
        del newdataset[s]
    print("len: ", len(newdataset))
    return newdataset

# DMP

In [15]:
%run DMP.ipynb
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
batch_amount = 100
# Generate Data using SEIRdata
graph_model = SIRdata(batches = batch_amount,show_progress=True, graph_type = "ER", iterations = 50, N=50 ,p=6/100, beta=1/20, gamma=1/50)
sirData = graph_model.generate_data()

Executing function generate_data :

0 %
10 %
20 %
30 %
40 %
50 %
60 %
70 %
80 %
90 %

Finished execution of function generate_data in 0.39409 seconds.



In [17]:
# Convert data to dtype data
sir_dataset = []
for graph_index in range(batch_amount):
    sir_dataset.append(Data(edge_index = sirData[graph_index][1], x = sirData[graph_index][2], y = sirData[graph_index][3]))
    sir_dataset[-1] = sir_dataset[-1].to(device)
del sirData

In [18]:
# test dmp
top10 = 0
top5 = 0
top1 = 0
total = 0
prog = 0

tester = dmp_layer(lamb=1/40,mu=1/100) 

for it, data in enumerate(sir_dataset):
    total +=1
    edge_index = data.edge_index
    snapshot = data.x
    
    prediction = tester.predict(edge_index, snapshot=snapshot, time_steps=100)
    
    correct = data.y.argmax().cpu().numpy()
    if correct in prediction.argsort()[-10:]:
        top10+=1
        if correct in prediction.argsort()[-5:]:
            top5+=1
            if correct == prediction.argmax():
                top1+=1

    if it/len(sir_dataset)*100>prog:
        print("prog", prog, "%")
        prog+=1
print("top10 accuracy: ", top10/total*100, "%")
print("top5 accuracy: ", top5/total*100, "%")
print("top1 accuracy: ", top1/total*100, "%")

prog 0 %
prog 1 %
prog 2 %
prog 3 %
prog 4 %
prog 5 %
prog 6 %
prog 7 %
prog 8 %
prog 9 %
prog 10 %
prog 11 %
prog 12 %
prog 13 %
prog 14 %
prog 15 %
prog 16 %
prog 17 %
prog 18 %
prog 19 %
prog 20 %
prog 21 %
prog 22 %
prog 23 %
prog 24 %
prog 25 %
prog 26 %
prog 27 %
prog 28 %
prog 29 %
prog 30 %
prog 31 %
prog 32 %
prog 33 %
prog 34 %
prog 35 %
prog 36 %
prog 37 %
prog 38 %
prog 39 %
prog 40 %
prog 41 %
prog 42 %
prog 43 %
prog 44 %
prog 45 %
prog 46 %
prog 47 %
prog 48 %
prog 49 %
prog 50 %
prog 51 %
prog 52 %
prog 53 %
prog 54 %
prog 55 %
prog 56 %
prog 57 %
prog 58 %
prog 59 %
prog 60 %
prog 61 %
prog 62 %
prog 63 %
prog 64 %
prog 65 %
prog 66 %
prog 67 %
prog 68 %
prog 69 %
prog 70 %
prog 71 %
prog 72 %
prog 73 %
prog 74 %
prog 75 %
prog 76 %
prog 77 %
prog 78 %
prog 79 %
prog 80 %
prog 81 %
prog 82 %
prog 83 %
prog 84 %
prog 85 %
prog 86 %
prog 87 %
prog 88 %
prog 89 %
prog 90 %
prog 91 %
prog 92 %
prog 93 %
prog 94 %
prog 95 %
prog 96 %
prog 97 %
prog 98 %
top10 accuracy:  75.